# Regression and other stories(Gelman et al. 2020) Earning data regression analysed directly through Stan code 

### Import Packages

In [5]:
import pystan as ps
import os
import pandas as pd
import numpy as np 
import arviz as az
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import requests
import io

### Import data

In [6]:
pwd = ("C:\\Users\\harri/OneDrive/Documents/Stats 2")
os.chdir(pwd)
# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/avehtari/ROS-Examples/master/Earnings/data/earnings.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe


### Data into pandas dataframe and total rows dropped as failing to intialise 

In [7]:
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df = df.drop(df.index[20:1800])
df.shape

(36, 15)

### Stan regression code

In [8]:
np.random.seed(101)

pooled_data = """
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] y;
}
"""
params = """
parameters {
    real alpha;
    real beta;
    real<lower=0> sigma;
}
"""
model = """
model{
    y ~ normal(alpha + beta * x, sigma);
}

"""

In [9]:
#Data put into numpy array for vectorisation
x = df["height"]
x = x.values
y = df["weight"]
y = y.values
type(y)

numpy.ndarray

In [10]:
# Put data into a dictionary
data = {'N': len(x), 'x': x, 'y': y}

# Compile the model
sm = ps.StanModel(model_code=pooled_data + params + model)

#Fit the model to the data
fit = sm.sampling(data=data, iter=1000, chains=4, warmup=500, thin=1, seed=101)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_379598f6382da7303ab98d6b177f9841 NOW.


In [12]:
fit